In [1]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl.query import Q
from fuzzywuzzy import fuzz
import re

In [2]:
books = [book.lower() for book in open('../files/books.txt').read().split('\n')]

In [3]:
client = Elasticsearch()
s = Search(using=client)

In [64]:
verse_parser = re.compile(r'(\d\s)?([\w\.]+)\s+([\d:,\-\s\;]+)')
def search(term):
    match = verse_parser.match(term)
    if match:
        response = s.query('match', verse=term).highlight('verse').execute()
        for hit in response:
            print(hit.verse, hit.meta.highlight.verse)
        return
    book_match = max(list(map(lambda book: fuzz.ratio(term.strip().lower(), book), books)))
    sorted_books = list(reversed(sorted(books, key=lambda book: fuzz.ratio(term.strip().lower(), book))))
    if book_match >= 60:
        book_query = Q('bool', must=[Q('match', book=sorted_books[0])])
        response = s.query(book_query).highlight('book').execute()
        for hit in response:
            print(' '.join(hit.meta.highlight.book).strip() + ' ' +  hit.verse.split()[-1])
        return
    else:
        query_string = Q(
            'match_phrase', text={'query': term, 'slop': 2})
        response = s.query(query_string).highlight('text').execute()
        for hit in response.hits.hits:
            print(' '.join(hit['highlight']['text']))
        return

In [67]:
search('Songs of S')

<em>Song</em> <em>of</em> <em>Solomon</em> 1:2
<em>Song</em> <em>of</em> <em>Solomon</em> 1:3
<em>Song</em> <em>of</em> <em>Solomon</em> 1:8
<em>Song</em> <em>of</em> <em>Solomon</em> 1:15
<em>Song</em> <em>of</em> <em>Solomon</em> 1:16
<em>Song</em> <em>of</em> <em>Solomon</em> 2:3
<em>Song</em> <em>of</em> <em>Solomon</em> 2:9
<em>Song</em> <em>of</em> <em>Solomon</em> 2:13
<em>Song</em> <em>of</em> <em>Solomon</em> 4:4
<em>Song</em> <em>of</em> <em>Solomon</em> 4:7
